# XGBoost #

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe 
from sklearn.metrics import  accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
solution = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [3]:
Y_train = train.claim
X_train = train.drop(columns = ['id' , 'claim'] , inplace = False)
test = test.drop(columns = ['id'] , inplace = False)

In [4]:
X_train['NaN_row'] = X_train.isna().sum(axis=1)
test['NaN_row'] = test.isna().sum(axis=1)

In [5]:
test['NaN_row'] = test.isna().sum(axis=1)
imputer = SimpleImputer(missing_values=np.nan, strategy='constant')
X_train = imputer.fit_transform(X_train)
test = imputer.transform(test)
scaling = StandardScaler()
X_train = scaling.fit_transform(X_train)
test = scaling.fit_transform(test)
X_train = pd.DataFrame(X_train)
test = pd.DataFrame(test)

In [6]:
def objective_xgb( trial):
    x_train , X_test , y_train , y_test = train_test_split(X_train , Y_train , random_state = 0 , test_size = 0.2)
    params = {'max_depth': hp.quniform("max_depth", 3, 10 , 2 ),
        'eta' : trial.suggest_loguniform('eta', 1e-5, 0.2),
        'gamma' : trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 1e4),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 1e4),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.1 , 1 ),
        'min_child_weight' : hp.quniform('min_child_weight', 100 , 1000 , 1),
        'subsample' : trial.suggest_float('subsample', 1e-8,1),
        'learning_rate': 0.01 ,
        'n_estimators': 10000,
        'seed': 0 ,
        'tree_method': 'gpu_hist' ,
         'gpu_id': 0, 
         'predictor': 'gpu_predictor'
    }
    eval_set = [(X_test, y_test)]
    model = xgb.XGBClassifier().set_params(**params)
    model.fit(x_train, y_train, early_stopping_rounds = 250, eval_set = eval_set, verbose = 500)
    y_pred = model.predict_proba(X_test)
    roc = roc_auc_score(pd.get_dummies(y_test), y_pred)
    return roc

In [7]:
X , X_val , Y , Y_val = train_test_split(X_train , Y_train , random_state = 0 , test_size = 0.05)

In [8]:
params = {
    'max_depth' : 8 , 'eta': 0.01476703340476827e-05, 'gamma': 0.02355394372959 ,
    'reg_lambda': 0.0016354050956302484, 'reg_alpha': 1.3209533717146242,'subsample': 0.9237998842768156 ,
    'colsample_bytree': 0.9, 'colsample_bylevel': 0.2 , 'booster': 'gbtree',
    'learning_rate':0.025 , 'n_estimators':10000,'seed': 0 ,'objective' : 'binary:logistic',
    'tree_method': 'gpu_hist' ,'gpu_id': 0, 'predictor': 'gpu_predictor' ,'min_child_weight': 250 
}

In [9]:
 model = xgb.XGBClassifier(**params , random_state=2021, use_label_encoder=False)
 model.fit(X , Y)
 Y_predict = model.predict(X_val)
 accuracy_score(Y_val , Y_predict)

[20:08:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7700434274260899

In [10]:
y_test = model.predict_proba(test)[:,1]
solution['claim'] = y_test
solution.to_csv('submission.csv', index=False)